# Домашнее задание №6. Нейронные сети
Суть этого задания в том, чтобы поиграться с параметрами сети, наблюдая за тем, как меняются результаты в зависимости от ваших действий.

## Задание 1
Скопируйте результаты первых трех заданий из Домашнего задания №5. У вас должен быть скачан и обработан датасет с данными о пассажирах Титаника. Получившийся DataFrame должен быть разделен на обучающую (80%) и тестовую выборку (20%).

In [ ]:
import pandas as pd

# Загрузка данных из файла train.csv
df = pd.read_csv('train.csv')

print(df.head())

In [ ]:
# Преобразование категориальных данных в числовые
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Удаление столбцов, которые не содержат числовых данных
df = df.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

# Обработка пропусков в числовых столбцах
df.fillna(df.mean(), inplace=True)

# Проверка на пропуски после обработки
print(df.isnull().sum())


In [ ]:
from sklearn.model_selection import train_test_split

# Целевая переменная (Survived)
y = df['Survived']

# Признаки (все столбцы, кроме Survived)
X = df.drop('Survived', axis=1)

# Разделение на обучающую (80%) и тестовую (20%) выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверка формы данных
print(f"Обучающая выборка X: {X_train.shape}")
print(f"Тестовая выборка X: {X_test.shape}")

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

# Инициализация модели перцептрона
model = Perceptron(max_iter=1000, random_state=42)

# Обучение модели на обучающих данных
model.fit(X_train, y_train)

# Прогнозирование на тестовых данных
y_pred = model.predict(X_test)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy:.2f}')

# Код, рассмотренный на семинаре

In [60]:
import numpy as np

#### Инициализация

In [61]:
def init_layers(nn_architecture, seed = 99):
    np.random.seed(seed)
    number_of_layers = len(nn_architecture)
    params_values = {}
    
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) * 0.1
        params_values['b' + str(layer_idx)] = np.random.randn(
            layer_output_size, 1) * 0.1
        
    return params_values

#### Функции активации

In [62]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

#### Прямой проход

In [63]:
def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    Z_curr = np.dot(W_curr, A_prev) + b_curr

    if activation == "relu":
        activation_func = relu
    elif activation == "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
        
    return activation_func(Z_curr), Z_curr

def full_forward_propagation(X, params_values, nn_architecture):
    memory = {}
    A_curr = X
    
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        A_prev = A_curr
        
        activ_function_curr = layer["activation"]
        W_curr = params_values["W" + str(layer_idx)]
        b_curr = params_values["b" + str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
        
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
       
    return A_curr, memory

#### Вычисление ошибки

In [64]:
def get_cost_value(Y_hat, Y):
    m = Y_hat.shape[1]
    cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

#### Обратный проход

In [65]:
def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    m = A_prev.shape[1]
    
    if activation == "relu":
        backward_activation_func = relu_backward
    elif activation == "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
    
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    
    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)
    
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));
    
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_idx_curr = layer_idx_prev + 1
        activ_function_curr = layer["activation"]
        
        dA_curr = dA_prev
        
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        
        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]
        
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)
        
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr
    
    return grads_values

#### Обновление весов

In [66]:
def update(params_values, grads_values, nn_architecture, learning_rate):

    for layer_idx, layer in enumerate(nn_architecture, 1):
        params_values["W" + str(layer_idx)] -= learning_rate * grads_values["dW" + str(layer_idx)]        
        params_values["b" + str(layer_idx)] -= learning_rate * grads_values["db" + str(layer_idx)]

    return params_values;

#### Оценка результата

In [67]:
def convert_prob_into_class(probs):
    probs_ = np.copy(probs)
    probs_[probs_ > 0.5] = 1
    probs_[probs_ <= 0.5] = 0
    return probs_

def get_accuracy_value(Y_hat, Y):
    Y_hat_ = convert_prob_into_class(Y_hat)
    return (Y_hat_ == Y).all(axis=0).mean()

#### Функция для обучения

In [68]:
def train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None):
    params_values = init_layers(nn_architecture, 2)
    cost_history = []
    accuracy_history = []
    
    for i in range(epochs):
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
        
        cost = get_cost_value(Y_hat, Y)
        cost_history.append(cost)
        accuracy = get_accuracy_value(Y_hat, Y)
        accuracy_history.append(accuracy)
        
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        
        if(i % 50 == 0):
            if(verbose):
                print("Iteration: {:05} - cost: {:.5f} - accuracy: {:.5f}".format(i, cost, accuracy))
            if(callback is not None):
                callback(i, params_values)
            
    return params_values

#### Ячейка с параметрами слоев

In [69]:
# NN_ARCHITECTURE = [
#     {"input_dim": 2, "output_dim": 25, "activation": "relu"},
#     {"input_dim": 25, "output_dim": 50, "activation": "relu"},
#     {"input_dim": 50, "output_dim": 50, "activation": "relu"},
#     {"input_dim": 50, "output_dim": 25, "activation": "relu"},
#     {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
# ]

#### Ячейка с обучением

In [70]:
# # Обучение
# params_values = train(np.transpose(X_train), np.transpose(y_train.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)
# # Предсказание
# Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)
# # Оценка на тестовой выборке
# acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.reshape((y_test.shape[0], 1))))
# print("Test set accuracy: {:.2f}".format(acc_test))

## Задание 2. Запуск на данных
Модифицируйте код в последних двух ячейках так, чтобы его можно было запускать на подготовленных данных. Запишите исправленные ячейки ниже и запустите.

In [71]:
NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]

In [ ]:
# Обучение
params_values = train(np.transpose(X_train), np.transpose(y_train.values.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 100, 0.01)
# Предсказание
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1))))
print("Test set accuracy: {:.2f}".format(acc_test))


## Задание 3. Нормализация данных
На результат обучения влияет то, что данные в разных столбцах могут принимать совсем разные по величине значения, все измеряются в разных единицах. Нормализуйте данные, например, при помощи <code>sklearn.preprocessing.StandardScaler</code>. А затем снова скопируйте сюда ячейку с обучением и запустите, чтобы оценить, как меняется результат.

In [73]:
# Напишите свой код в данной ячейке
from sklearn.preprocessing import StandardScaler
# Нормализация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Задание 4. Число эпох
Попробуйте обучать, используя разные значения числа эпох.

In [ ]:
# Ячейка с обучением, в которой число эпох равно 10
# Обучение с разным числом эпох

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=10,
    learning_rate=0.01,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))


In [ ]:
# Ячейка с обучением, в которой число эпох равно 1000

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=1000,
    learning_rate=0.01,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))

In [ ]:
# Ячейка с обучением, в которой число эпох равно 10000

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=10000,
    learning_rate=0.01,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))

## Задание 5. Скорость обучения
Попробуйте обучать, используя разные значения скорости обучения. Число эпох выберите на выше усмотрение, но одинаковое во всех трех ячейках.

In [ ]:
# Ячейка с обучением, в которой скорость обучения равна 0.0001

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=1000,
    learning_rate=0.0001,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))

In [ ]:
# Ячейка с обучением, в которой скорость обучения равна 0.001

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=1000,
    learning_rate=0.001,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))

In [ ]:
# Ячейка с обучением, в которой скорость обучения равна 0.01

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=1000,
    learning_rate=0.01,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))

## Задание 6. Изменение слоев
Начиная с этого задания вы можете выбирать число эпох и скорость обучения самостоятельно.

А еще теперь кроме ячейки с обучением будет меняться еще и ячейка с параметрами слоев. То есть каждая ячейка с ответом будет выглядеть как-то так:

In [ ]:
NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=1000,
    learning_rate=0.01,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))

После каждого изменения параметров, не забывайте вставлять код с обучением и выводом accuracy.

In [ ]:
# Ячейка с обучением и параметрами слоев, в которой есть только последний выходной слой с sigmoid

# Какой-то комментарий про эту ячейку
NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 25, "activation": "sigmoid"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=1000,
    learning_rate=0.01,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))

In [ ]:
# Ячейка с обучением и параметрами слоев, в которой все функции активации это sigmoid

NN_ARCHITECTURE = [
    {"input_dim": 7, "output_dim": 25, "activation": "sigmoid"},
    {"input_dim": 25, "output_dim": 50, "activation": "sigmoid"},
    {"input_dim": 50, "output_dim": 50, "activation": "sigmoid"},
    {"input_dim": 50, "output_dim": 25, "activation": "sigmoid"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=1000,
    learning_rate=0.01,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))

In [ ]:
# Ячейка с обучением и параметрами слоев, в которой есть два слоя с relu и выходной слой с sigmoid
NN_ARCHITECTURE_RELU_SIGMOID = [
    {"input_dim": 7, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 1, "activation": "sigmoid"},
]

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=1000,
    learning_rate=0.01,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))

In [ ]:
# Ячейка с обучением и параметрами слоев, в которой есть два слоя с relu и выходной слой с sigmoid
# а еще число нейронов на каждом слое равно 8 (кроме входного и выходного)
NN_ARCHITECTURE_TWO_RELU_8 = [
    {"input_dim": 2, "output_dim": 8, "activation": "relu"},
    {"input_dim": 8, "output_dim": 8, "activation": "relu"},
    {"input_dim": 8, "output_dim": 1, "activation": "sigmoid"},
]

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=1000,
    learning_rate=0.01,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))

In [ ]:
# Ячейка с обучением и параметрами слоев, в которой есть два слоя с relu и выходной слой с sigmoid
# а еще число нейронов на каждом слое равно 100 (кроме входного и выходного)
NN_ARCHITECTURE_TWO_RELU_100 = [
    {"input_dim": 2, "output_dim": 100, "activation": "relu"},
    {"input_dim": 100, "output_dim": 100, "activation": "relu"},
    {"input_dim": 100, "output_dim": 1, "activation": "sigmoid"},
]

# Обучение
params_values = train(
    np.transpose(X_train),
    np.transpose(y_train.values.reshape((y_train.shape[0], 1))),
    NN_ARCHITECTURE,
    epochs=1000,
    learning_rate=0.01,
)
# Предсказание
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE
)
# Оценка на тестовой выборке
acc_test = get_accuracy_value(
    Y_test_hat, np.transpose(y_test.values.reshape((y_test.shape[0], 1)))
)
print("Test set accuracy: {:.2f}".format(acc_test))